This program filters the WRDS quarterly fundamentals dataset, removing firms that 

1. only have revenue values of `NaN`.

The program also adds observations such that a firm will have rows for all quarters between its first and last time in the dataset. 


**EDIT (2024-02-05):** I don't know if the above is entirely true. I do fill in the missing quarters, but I don't think I filter for `na` revenue values. I have not changed any of the code since 2023-12-27.

**EDIT (2024-02-05):** Actually, I do see now that I do filter the `na` revenue values. I have not changed any of the code since 2023-12-27.

**EDIT (2024-02-05):** I added one line to see the number of things in `wrds` before any filtering. 

# Setup

In [1]:
import numpy as np
import pandas as pd
import datetime
import time

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa as tsa
from statsmodels.regression.rolling import RollingOLS

In [2]:
wrds = pd.read_csv('../Data/Thesis_Fundamentals_Data_1960-2023.csv',
                   dtype={
                       'gvkey': np.unicode_,
                       'datadate': np.unicode_,
                       'fyearq': 'float32',
                       'fqtr': 'float32',
                       'indfmt': np.unicode_,
                       'consol': np.unicode_,
                       'popsrc': np.unicode_,
                       'datafmt': np.unicode_,
                       'tic': np.unicode_,
                       'cusip': np.unicode_,
                       'conm': np.unicode_,
                       'curcdq': np.unicode_,
                       'datacqtr': np.unicode_,
                       'datafqtr': np.unicode_,
                       'naics': np.unicode_,
                       'sic': np.unicode_,
                       'ggroup': np.unicode_,
                       'gind': np.unicode_,
                       'gsector': np.unicode_,
                       'gsubind': np.unicode_,
                       'cik': np.unicode_,
                       'costat': np.unicode_,
                       'actq': 'float64',
                       'atq': 'float64',
                       'ceqq': 'float64',
                       'chq': 'float64',
                       'cshfdq': 'float64',
                       'cshoq': 'float64',
                       'cshprq': 'float64',
                       'dlcq': 'float64',
                       'dlttq': 'float64',
                       'dpactq': 'float64',
                       'dpq': 'float64',
                       'icaptq': 'float64',
                       'lctq': 'float64',
                       'niq': 'float64',
                       'oiadpq': 'float64',
                       'oibdpq': 'float64',
                       'piq': 'float64',
                       'revtq': 'float64',
                       'teqq': 'float64',
                       'txtq': 'float64',
                       'xintq': 'float64',
                       'xrdq': 'float64',
                       'dvy': 'float64',
                       'cik': 'float64',
                       'mktvalq': 'float64',
                       'prccq': 'float64'
                  })
wrds['datadate'] = pd.to_datetime(wrds.datadate)

print(len(wrds.gvkey.unique()), 'Firms')

36775 Firms


In [3]:
wrds

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,tic,cusip,...,cik,costat,mkvaltq,prccq,ggroup,gind,gsector,gsubind,naics,sic
0,001000,1966-03-31,1966.0,1.0,INDL,C,D,STD,AE.2,000032102,...,NaN,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3089
1,001000,1966-06-30,1966.0,2.0,INDL,C,D,STD,AE.2,000032102,...,NaN,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3089
2,001000,1966-09-30,1966.0,3.0,INDL,C,D,STD,AE.2,000032102,...,NaN,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3089
3,001000,1966-12-31,1966.0,4.0,INDL,C,D,STD,AE.2,000032102,...,NaN,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3089
4,001000,1967-03-31,1967.0,1.0,INDL,C,D,STD,AE.2,000032102,...,NaN,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1774461,353945,2022-09-30,2022.0,3.0,INDL,C,D,STD,ACLLY,00449R109,...,1948862.0,A,NaN,NaN,2010,201040,20,20104020,335312,3621
1774462,353945,2022-12-31,2022.0,4.0,INDL,C,D,STD,ACLLY,00449R109,...,1948862.0,A,NaN,20.6930,2010,201040,20,20104020,335312,3621
1774463,354003,2023-03-31,2023.0,1.0,INDL,C,D,STD,BEMB,46436E262,...,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,525910,6722
1774464,354003,2023-06-30,2023.0,2.0,INDL,C,D,STD,BEMB,46436E262,...,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,525910,6722


|Key|Accounting Term|
|:---|---:|
|gvkey|S&P's Code|
|datadate|Quarter Ending Date|
|fyearq|Fiscal Year|
|fqtr|Fiscal Quarter|
|indfmt|Industrials Finances Format|
|consol|Subsidiaries Consolidated|
|popsrc|Domestic/International|
|datafmt|Standardized Quarterly Data|
|tic|Ticker (Symbol)|
|conm|Company Name|
|curcdq|Currency (USD)|
|datacqtr|Calendar Quarter|
|datafqtr|Fiscal Quarter|
|atq|Total Assets|
|chq|Cash|
|cshfdq|Diluted Shares|
|cshoq|Outstanding Shares|
|cshprq|Basic Shares|
|dlttq|Total Long-Term Debt|
|dpactq|Quarterly Depreciation|
|icaptq|Invested Cpaital|
|oiadpq|Quarterly Operating Income|
|revtq|Quarterly Revenue|
|costat|Company Active/Inactive|
|prccq|Stock Price at Quarter End|
|naics|NAICS Industry Code|

In [3]:
wrds = wrds[(wrds.revtq.isna() == False) & (wrds.datacqtr.isnull() == False)].copy()
#wrds = wrds[wrds.revtq > 0].copy()
#wrds = wrds[abs(wrds.oiadpq / wrds.revtq) < 10].copy()
wrds

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,tic,cusip,...,cik,costat,mkvaltq,prccq,ggroup,gind,gsector,gsubind,naics,sic
3,001000,1966-12-31,1966.0,4.0,INDL,C,D,STD,AE.2,000032102,...,NaN,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3089
7,001000,1967-12-31,1967.0,4.0,INDL,C,D,STD,AE.2,000032102,...,NaN,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3089
11,001000,1968-12-31,1968.0,4.0,INDL,C,D,STD,AE.2,000032102,...,NaN,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3089
12,001000,1969-03-31,1969.0,1.0,INDL,C,D,STD,AE.2,000032102,...,NaN,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3089
13,001000,1969-06-30,1969.0,2.0,INDL,C,D,STD,AE.2,000032102,...,NaN,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1774452,353444,2023-03-31,2023.0,1.0,INDL,C,D,STD,HLN,405552100,...,1900304.0,A,NaN,8.140,3030,303020,30,30302010,325412,2834
1774453,353444,2023-06-30,2023.0,2.0,INDL,C,D,STD,HLN,405552100,...,1900304.0,A,NaN,8.380,3030,303020,30,30302010,325412,2834
1774454,353444,2023-09-30,2023.0,3.0,INDL,C,D,STD,HLN,405552100,...,1900304.0,A,NaN,8.330,3030,303020,30,30302010,325412,2834
1774458,353945,2021-12-31,2021.0,4.0,INDL,C,D,STD,ACLLY,00449R109,...,1948862.0,A,NaN,NaN,2010,201040,20,20104020,335312,3621


    relevant_list = [
        'gvkey',
        'datadate',
        'datafqtr',
        'datacqtr',
        'conm',
        'revtq',
        'mkvaltq',
        'chq',
        'oiadpq'
    ]

    companykey = '026097'
    companyyear = 2019

    wrds[(wrds.gvkey == '026097') & 
         (wrds.datadate < np.datetime64(str(companyyear + 1) + '-01-01')) & 
         (wrds.datadate >= np.datetime64(str(companyyear) + '-01-01'))][relevant_list]

In [4]:
wrds['qid'] = wrds.apply(lambda row: row.gvkey + '_' + row.datacqtr, axis=1)

In [5]:
wrds['tid'] = wrds.apply(lambda row: row.gvkey + '_' + str(row.datadate.date()), axis=1)

In [6]:
wrds.gvkey.nunique()

28721

In [7]:
fast_df = wrds[['gvkey', 'datacqtr']].copy()

In [9]:
qids_lst = []

for i, company in enumerate(fast_df.gvkey.unique()):
    rel_series = fast_df[fast_df.gvkey == company].datacqtr.sort_values(ascending=True)
    start = rel_series.iloc[0]
    end = rel_series.iloc[-1]
    ey = int(end[:4])
    eq = int(end[-1])
    current = start
    y = int(start[:4])
    q = int(start[-1])
    qids_lst.append(company + '_' + current)
    while (current != end) and not (y >= ey and q >= eq):
        q += 1
        if q > 4:
            q = 1
            y += 1
        current = str(y) + 'Q' + str(q)
        qids_lst.append(company + '_' + current)
        if y >= 2024:
            break
    #if i >= 50: 
    #    break
    

In [10]:
wrds.set_index('qid', inplace=True)

In [11]:
new_wrds = pd.DataFrame(index=qids_lst)
new_wrds = pd.merge(new_wrds, wrds, left_index=True, right_index=True, how='outer')

In [12]:
new_wrds['qid'] = new_wrds.index
new_wrds['gvkey'] = new_wrds.qid.apply(lambda entry: entry[:6])
new_wrds['datacqtr'] = new_wrds.qid.apply(lambda entry: entry[-6:])

In [13]:
original_cols = new_wrds.columns
idcols = ['cusip', 'tic', 'datafmt', 'popsrc', 'consol', 'sic', 'naics']
new_wrds = pd.merge(
    new_wrds, 
    wrds.groupby('gvkey').first()[idcols], 
    left_on='gvkey', 
    right_index=True, 
    suffixes=('', '_fromgroup')
)
new_wrds

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,tic,cusip,...,sic,tid,qid,cusip_fromgroup,tic_fromgroup,datafmt_fromgroup,popsrc_fromgroup,consol_fromgroup,sic_fromgroup,naics_fromgroup
001000_1966Q4,001000,1966-12-31,1966.0,4.0,INDL,C,D,STD,AE.2,000032102,...,3089,001000_1966-12-31,001000_1966Q4,000032102,AE.2,STD,D,C,3089,None
001000_1967Q1,001000,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,001000_1967Q1,000032102,AE.2,STD,D,C,3089,None
001000_1967Q2,001000,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,001000_1967Q2,000032102,AE.2,STD,D,C,3089,None
001000_1967Q3,001000,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,001000_1967Q3,000032102,AE.2,STD,D,C,3089,None
001000_1967Q4,001000,1967-12-31,1967.0,4.0,INDL,C,D,STD,AE.2,000032102,...,3089,001000_1967-12-31,001000_1967Q4,000032102,AE.2,STD,D,C,3089,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353945_2021Q4,353945,2021-12-31,2021.0,4.0,INDL,C,D,STD,ACLLY,00449R109,...,3621,353945_2021-12-31,353945_2021Q4,00449R109,ACLLY,STD,D,C,3621,335312
353945_2022Q1,353945,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,353945_2022Q1,00449R109,ACLLY,STD,D,C,3621,335312
353945_2022Q2,353945,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,353945_2022Q2,00449R109,ACLLY,STD,D,C,3621,335312
353945_2022Q3,353945,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,353945_2022Q3,00449R109,ACLLY,STD,D,C,3621,335312


In [14]:
for col in idcols:
    new_wrds[col].fillna(new_wrds[col + '_fromgroup'], inplace=True)
new_wrds = new_wrds[original_cols]

In [15]:
new_wrds

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,tic,cusip,...,mkvaltq,prccq,ggroup,gind,gsector,gsubind,naics,sic,tid,qid
001000_1966Q4,001000,1966-12-31,1966.0,4.0,INDL,C,D,STD,AE.2,000032102,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3089,001000_1966-12-31,001000_1966Q4
001000_1967Q1,001000,NaT,NaN,NaN,NaN,C,D,STD,AE.2,000032102,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3089,NaN,001000_1967Q1
001000_1967Q2,001000,NaT,NaN,NaN,NaN,C,D,STD,AE.2,000032102,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3089,NaN,001000_1967Q2
001000_1967Q3,001000,NaT,NaN,NaN,NaN,C,D,STD,AE.2,000032102,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3089,NaN,001000_1967Q3
001000_1967Q4,001000,1967-12-31,1967.0,4.0,INDL,C,D,STD,AE.2,000032102,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3089,001000_1967-12-31,001000_1967Q4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353945_2021Q4,353945,2021-12-31,2021.0,4.0,INDL,C,D,STD,ACLLY,00449R109,...,NaN,NaN,2010,201040,20,20104020,335312,3621,353945_2021-12-31,353945_2021Q4
353945_2022Q1,353945,NaT,NaN,NaN,NaN,C,D,STD,ACLLY,00449R109,...,NaN,NaN,NaN,NaN,NaN,NaN,335312,3621,NaN,353945_2022Q1
353945_2022Q2,353945,NaT,NaN,NaN,NaN,C,D,STD,ACLLY,00449R109,...,NaN,NaN,NaN,NaN,NaN,NaN,335312,3621,NaN,353945_2022Q2
353945_2022Q3,353945,NaT,NaN,NaN,NaN,C,D,STD,ACLLY,00449R109,...,NaN,NaN,NaN,NaN,NaN,NaN,335312,3621,NaN,353945_2022Q3


In [17]:
new_wrds.to_csv('../Data/Thesis_Fundamentals_Data_1960-2023_FILTERED.csv')